In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%reload_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import sys
sys.path.append('../src')

import cv2 as cv
from cellpose import models, io, plot

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pickle
import re
from time import time
from tqdm import tqdm
from utils import list_subdir_filter as lsd#, slide_cycle_dot as scd
from metadata import metadata

md = metadata()

In [3]:
img_files =  lsd(md.folders['bg_removed'], True, '\.npy')
out_dir = os.path.join(md.folders['segmented'], 'cellpose')
Path(out_dir).mkdir(exist_ok=True, parents=True)

model = models.Cellpose(gpu=True, model_type='nuclei')

In [6]:
import pytorch.cuda

ModuleNotFoundError: No module named 'pytorch'

In [4]:
img_files[2:3]

['/projects/ag-bozek/lunaphore/data/interim/bg_removed/A40.2294_bg_removed_uint8.npy']

In [5]:
%%time
for img_file in tqdm(img_files[2:3]):
    tissue_id =re.sub('.*A40\.([0-9]{4}).*$', '\\1', img_file)
    img = np.load(img_file)[0]
    max_size = np.shape(img)[0]
    width = 3000
    for i in range((max_size // width) + 1):
        print(i)
        min_row, max_row = max(0, i * width - 100), min(max_size, (i+1) * width + 100)
        masks, flows, styles, diams = model.eval(img[min_row:max_row], diameter=None, channels=[0,0])
        pickle_dict = {'masks':masks, 'flows':flows, 'styles':styles, 'diams':diams}
        np.save(f'T3_A40_{tissue_id}_dapi1_masks_{i:02}.npy', masks)
        with open(os.path.join(out_dir, f'T3_A40_{tissue_id}_dapi1_{i:02}.pkl'), 'wb') as handle:
            pickle.dump(pickle_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


# it was 42 minutes, let's see now

  0%|                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

0


  0%|                                                                                                                                                 | 0/1 [01:27<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 284.00 MiB (GPU 0; 31.75 GiB total capacity; 2.62 GiB already allocated; 203.19 MiB free; 2.80 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [14]:
%%time
masks, flows, styles, diams = model.eval(ii[:, :3300], diameter=None, channels=[0,0])

CPU times: user 1min 12s, sys: 26.6 s, total: 1min 39s
Wall time: 1min 38s


In [ ]:
%%time
with open(os.path.join(out_dir, f'test.pkl'), 'wb') as handle:
    pickle.dump(masks, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
%%time
np.save(os.path.join(out_dir, f'test.pkl'), masks)

CPU times: user 2.7 ms, sys: 42.4 ms, total: 45.1 ms
Wall time: 298 ms


In [30]:
%%time
# masks, flows, styles, diams = model.eval(img[i*3], diameter=None, channels=[0,0])
pickle_dict = {'masks':masks, 'flows':flows, 'styles':styles, 'diams':diams}
with open(os.path.join(out_dir, f'A.pkl'), 'wb') as handle:
    pickle.dump(pickle_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

CPU times: user 0 ns, sys: 668 ms, total: 668 ms
Wall time: 1.24 s


In [ ]:
12096 / 32

In [ ]:
unit = 12096 / 2
measures = []
for i in range(2):
    measures.append([int(max(unit * i - unit/8, 0)), int(min(unit * (i+1) + unit/8, 12096))])

In [ ]:
for img_file in tqdm(img_files):
    tissue_id =re.sub('.*A40\.([0-9]{4}).*$', '\\1', img_file)
    img = np.load(img_file)
    for i in range(4):
        try:
            masks, flows, styles, diams = model.eval(img[i*3], diameter=None, channels=[0,0])
            pickle_dict = {'masks':masks}, 'flows':flows, 'styles':styles, 'diams':diams}
            with open(os.path.join(out_dir, f'A40_{tissue_id}_dapi{i}.pkl'), 'wb') as handle:
                pickle.dump(masks, handle, protocol=pickle.HIGHEST_PROTOCOL)
        except:
            print(f'Error with sample A40.{tissue_id}_dapi{i}')
        # for j in range(2):
        #     for k in range(2):
        #         to_process = img[i, slice(*measures[j]), slice(*measures[k])]
        #         masks, flows, styles, diams = model.eval(to_process, diameter=None, channels=[0,0])
        #         pickle_dict = {'masks':masks, 'flows':flows, 'styles':styles, 'diams':diams}
        #         with open(os.path.join(out_dir, f'slide{slide}_dot{dot}_dapi{i}_tile{j}_{k}.pkl'), 'wb') as handle:
        #             pickle.dump(pickle_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%%time
fig = plt.figure(figsize=(20,5))
plot.show_segmentation(fig, im1[0], masks, flows[0], channels=[0,0])
plt.tight_layout()
plt.show()

In [ ]:
np.shape(flows[0][::10, ::10])

In [ ]:
plt.imshow(crop_img[1], cmap='gray')